# General Setup

In [50]:
import json
import pandas as pd
import subprocess
import os

In [51]:
with open('data/curated/player_data.json', 'r') as f:
	player_data = json.load(f)

In [52]:
df = pd.DataFrame(player_data)

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15437 entries, 14216 to 10673
Data columns (total 46 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   match_id                  15437 non-null  int64  
 1   match_start_time          15437 non-null  int64  
 2   account_id                15437 non-null  int64  
 3   name                      15437 non-null  object 
 4   fantasy_role              15437 non-null  object 
 5   hero_id                   15437 non-null  int64  
 6   hero_variant              15437 non-null  object 
 7   isRadiant                 15437 non-null  bool   
 8   radiant_team_id           15437 non-null  int64  
 9   dire_team_id              15437 non-null  int64  
 10  tournament_start_date     15437 non-null  object 
 11  duration                  15437 non-null  int64  
 12  win                       15437 non-null  int64  
 13  kills                     15437 non-null  int64  
 14  deaths 

In [54]:
df = df.astype({
    'match_id': 'string',
    'account_id': 'string',
    'hero_id': 'string',
    'radiant_team_id': 'string',
    'dire_team_id': 'string',
    'win': 'boolean',
    'firstblood_claimed': 'boolean',
    'team_id': 'string',
	'enemy_team_id': 'string',
    'valveId': 'string',
})

## Set missing glicko scores to the first quartile
*Using the "..._clean" data set, this does not affect anything.*
Q1 was chosen, because teams that do not have a glicko 2 rating, usually are new or weaker teams.

In [55]:
q1_glicko = df['glicko2_rating'].quantile(0.25)
df['glicko2_rating'] = df['glicko2_rating'].fillna(q1_glicko)
q1_enemy_glicko = df['enemy_glicko2_rating'].quantile(0.25)
df['enemy_glicko2_rating'] = df['enemy_glicko2_rating'].fillna(q1_enemy_glicko)

In [56]:
mask = df['rating_advantage'].isna()
df.loc[mask, 'rating_advantage'] = df.loc[mask, 'glicko2_rating'] - df.loc[mask, 'enemy_glicko2_rating']

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15437 entries, 14216 to 10673
Data columns (total 46 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   match_id                  15437 non-null  string 
 1   match_start_time          15437 non-null  int64  
 2   account_id                15437 non-null  string 
 3   name                      15437 non-null  object 
 4   fantasy_role              15437 non-null  object 
 5   hero_id                   15437 non-null  string 
 6   hero_variant              15437 non-null  object 
 7   isRadiant                 15437 non-null  bool   
 8   radiant_team_id           15437 non-null  string 
 9   dire_team_id              15437 non-null  string 
 10  tournament_start_date     15437 non-null  object 
 11  duration                  15437 non-null  int64  
 12  win                       15437 non-null  boolean
 13  kills                     15437 non-null  int64  
 14  deaths 

# XGBoost
Without time dimension.

In [58]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error


In [59]:
enemy_team_avg = df.groupby('enemy_team_id')['fantasy_score'].mean().rename('enemy_avg_fantasy_score')
df = df.merge(enemy_team_avg, left_on='enemy_team_id', right_index=True, how='left')

In [60]:
not_features = [
	'match_id', 'name', 'match_start_time', 'radiant_team_id', 'dire_team_id',
	'tournament_start_date', 'teamName', 'valveId', 'enemy_teamName', 'enemy_valveId', 'fantasy_score',
    'rolling_winrate_10',
    'rolling_winrate_15',
    'rolling_fantasy_score_10',
    'rolling_fantasy_score_15',
    'rolling_enemy_glicko2_10',
    'rolling_enemy_glicko2_15',
]

In [61]:
not_features_fe = [
    'match_id', 'name', 'match_start_time', 'radiant_team_id', 'dire_team_id',
    'tournament_start_date', 'teamName', 'valveId', 'enemy_teamName', 'enemy_valveId', 'fantasy_score', 
]

# XHBoost without feature engineering

In [62]:
X = df.drop(columns=not_features)
y = df['fantasy_score']

In [63]:
X = pd.get_dummies(X, columns=['fantasy_role', 'hero_id', 'hero_variant', 'account_id', 'team_id', 'enemy_team_id'])


In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1001)


In [65]:
model = xgb.XGBRegressor()
model.fit(X_train, y_train)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [66]:
y_pred = model.predict(X_test)
print("RMSE:", root_mean_squared_error(y_test, y_pred))

RMSE: 0.8673414545412719


# XGBoost with Feature Engineering

In [67]:
X_fe = df.drop(columns=not_features_fe)
y_fe = df['fantasy_score']

In [68]:
X_fe = pd.get_dummies(X_fe, columns=['fantasy_role', 'hero_id', 'hero_variant', 'account_id', 'team_id', 'enemy_team_id'])

In [69]:
X_fe_train, X_fe_test, y_fe_train, y_fe_test = train_test_split(X_fe, y_fe, test_size=0.20, random_state=1001)

In [70]:
model = xgb.XGBRegressor()
model.fit(X_fe_train, y_fe_train)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [71]:
y_fe_pred = model.predict(X_fe_test)
print("RMSE:", root_mean_squared_error(y_fe_test, y_fe_pred))

RMSE: 0.8776266747620363


# Prediction Framework

In [72]:
def build_prediction_rows(df, matchups, lineup_dir, rolling_n=20):
    rows = []
    for team_id, enemy_team_id in matchups:
        lineup_path = os.path.join(lineup_dir, f"team_lineup_{team_id}.json")
        if not os.path.exists(lineup_path):
            subprocess.run(["python", "get_current_lineups.py", str(team_id)])
        if not os.path.exists(lineup_path):
            continue  # Skip if still missing
        with open(lineup_path, "r", encoding="utf-8") as f:
            lineup = [p for p in json.load(f) if p.get("is_current_team_member")]
        for player in lineup:
            account_id = str(player["account_id"])
            player_hist = df[df["account_id"] == account_id].sort_values("match_start_time")
            if player_hist.empty:
                continue
            numeric_cols = player_hist.select_dtypes(include='number').columns.difference(
                ['match_id', 'account_id', 'radiant_team_id', 'dire_team_id', 'team_id', 'enemy_team_id', 'match_start_time']
            )
            rolling = player_hist[numeric_cols].tail(rolling_n).mean()
            row = player_hist.iloc[-1].copy()
            row[numeric_cols] = rolling
            row["team_id"] = team_id
            row["enemy_team_id"] = enemy_team_id
            rows.append(row)
    return pd.DataFrame(rows)

In [73]:
def predict_players_for_matchups(df, model, matchup_list, X, not_features, lineup_dir="data/lineups", rolling_n=20):
    # Ensure all lineup files exist (call API if not)
    team_ids = set()
    for t1, t2 in matchup_list:
        team_ids.add(t1)
        team_ids.add(t2)
    for team_id in team_ids:
        lineup_path = os.path.join(lineup_dir, f"team_lineup_{team_id}.json")
        if not os.path.exists(lineup_path):
            subprocess.run(["python", "get_current_lineups.py", str(team_id)])
    
    # Build synthetic prediction rows
    pred_df = build_prediction_rows(df, matchup_list, lineup_dir, rolling_n=rolling_n)
    if pred_df.empty:
        print("No eligible players found for the given matchups.")
        return pd.DataFrame()
    # Prepare features (same as training)
    X_pred = pred_df.drop(columns=not_features)
    X_pred = pd.get_dummies(X_pred, columns=['fantasy_role', 'hero_id', 'hero_variant', 'account_id', 'team_id', 'enemy_team_id'])
    X_pred = X_pred.reindex(columns=X.columns, fill_value=0)
    # print(X_pred.filter(like='enemy_team_id').head(10)) # Debugger, shows the dummy encoded enemy_team variable.
    # More debugging
    #rows = X_pred[X_pred['account_id_847565596'] == 1]  # print dummies for "rue"
    #print(rows[['enemy_team_id_7119388', 'enemy_team_id_8261500', 'enemy_team_id_8597976']])
    
    pred_df['predicted_fantasy_score'] = model.predict(X_pred) # Predict fantasy scores
    return pred_df[['account_id', 'name', 'fantasy_role', 'team_id', 'enemy_team_id', 'predicted_fantasy_score']]

In [74]:
def make_symmetric_matchups(matchups):
    symmetric = []
    for t1, t2 in matchups:
        symmetric.append((t1, t2))
        symmetric.append((t2, t1))
    return symmetric

In [75]:
matchups = [('7119388', '8261500'), ('8597976', '7119388'), ('8597976', '8261500')] # Spirit vs. Xtreme Gaming, Spirit vs. Talon
symetric_matchups = make_symmetric_matchups(matchups)
result = predict_players_for_matchups(df, model, symetric_matchups, X, not_features)

ValueError: feature_names mismatch: ['isRadiant', 'duration', 'win', 'kills', 'deaths', 'last_hits', 'denies', 'gold_per_min', 'towers_killed', 'roshans_killed', 'teamfight_participation', 'obs_placed', 'camps_stacked', 'rune_pickups', 'firstblood_claimed', 'stuns', 'creeps_stacked', 'assists', 'xp_per_min', 'lane_efficiency', 'glicko2_rating', 'enemy_glicko2_rating', 'rating_advantage', 'rolling_winrate_10', 'rolling_winrate_15', 'rolling_fantasy_score_10', 'rolling_fantasy_score_15', 'rolling_enemy_glicko2_10', 'rolling_enemy_glicko2_15', 'enemy_avg_fantasy_score', 'fantasy_role_Core', 'fantasy_role_Mid', 'fantasy_role_Support', 'hero_id_1', 'hero_id_10', 'hero_id_100', 'hero_id_101', 'hero_id_102', 'hero_id_103', 'hero_id_104', 'hero_id_105', 'hero_id_106', 'hero_id_107', 'hero_id_108', 'hero_id_109', 'hero_id_11', 'hero_id_110', 'hero_id_111', 'hero_id_112', 'hero_id_113', 'hero_id_114', 'hero_id_119', 'hero_id_12', 'hero_id_120', 'hero_id_121', 'hero_id_123', 'hero_id_126', 'hero_

In [ ]:
result

In [ ]:
import matplotlib.pyplot as plt
xgb.plot_importance(model)
plt.show()
feature_importances = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
})

In [ ]:
feature_importances = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
})
feature_importances = feature_importances.sort_values(by='importance', ascending=False)
feature_importances

testing stuff

In [ ]:
print(df['enemy_glicko2_rating'].describe())
print(df['enemy_glicko2_rating'].isna().sum())

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor


In [ ]:
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X, y, test_size=0.2)

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=1001)
rf_model.fit(X_train_rf, y_train_rf)

In [ ]:
y_pred_rf = rf_model.predict(X_test_rf)
print("Random Forest RMSE:", root_mean_squared_error(y_test_rf, y_pred_rf))

In [ ]:
matchups = [('7119388', '8261500'), ('8597976', '7119388'), ('8597976', '8261500')] # Spirit vs. Xtreme Gaming, Spirit vs. Talon
symetric_matchups = make_symmetric_matchups(matchups)
result = predict_players_for_matchups(df, rf_model, symetric_matchups, X, not_features)

In [ ]:
result

In [ ]:
rf_feature_importances = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values(by='importance', ascending=False)
rf_feature_importances

# LSTM
Wegen den Python-Version voraussetzungen von TensorFlow wird LSTM in einem eigenen Notebook implementiert.